In [1]:
import re
import time
from datetime import datetime
import numpy as np
import pandas as pd
import lxml.html
import urllib
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains # for advance action (drag, drop)
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
time.strftime("%Y-%m-%d %H:%M:%S")

'2023-02-04 07:44:10'

In [3]:
a = [1,2,3,33,4,5]
if 3 in a or 33 in a:
    print('yes')
else:
    print('no')

yes


In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

pathDriver = 'Chrome/chromedriver.exe'
url = 'https://www.google.com/maps/place/National+Library+of+the+Republic+of+Indonesia/@-6.1811773,106.8247317,17z/data=!4m7!3m6!1s0x2e69f442596e0c93:0x4ba58be40979fe36!8m2!3d-6.1809493!4d106.8269234!9m1!1b1'
driver = webdriver.Chrome(executable_path=pathDriver, options=options)
driver.get(url)

# klik tombol sort untuk sortir
menu_bt = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(
                       (By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button'))
                   )  
menu_bt.click()
time.sleep(1)

# pilih Newest untuk mengurutkan ulasan dari yang terbaru
recent_rating_bt = driver.find_elements_by_xpath('//*[@id="action-menu"]/div[2]')[0]
recent_rating_bt.click()
time.sleep(5)

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

number = 0
stop_scroll_eng = ['1 years ago','2 years ago'] # '3 months ago','6 months ago','9 months ago',
stop_scroll_ina = ['1 tahun lalu','2 tahun lalu'] # '3 bulan lalu','6 bulan lalu','9 bulan lalu',
for i in range(len(stop_scroll_eng)):
    print('Section', stop_scroll_ina[i])
    while True:
        number += 1

        # Scroll down to bottom
        ele = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
        driver.execute_script('arguments[0].scrollBy(0, 10000);', ele)

        # Wait to load page
        time.sleep(5)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return arguments[0].scrollHeight", ele)

        # find a specific date to stop from unlimited scroll
        filter_date = driver.find_elements_by_class_name('rsqaWe')
        filters = []
        for fil in filter_date:
            filters.append(fil.text)

        if stop_scroll_eng[i] in filters or stop_scroll_ina[i] in filters: #'2 months ago':
            break
    #     if number == 5:
    #         break
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(1)
    print('scraping proccess')
    
    # click more or lainnya
    item = driver.find_elements_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]')
    for i in item:
        button = i.find_elements_by_tag_name('button')
        for m in button:
            if m.text == "Lainnya":
                m.click()
    time.sleep(4)


    # scrape html
    response = BeautifulSoup(driver.page_source, 'html.parser')
    review_list = response.find_all('div', {'class':'jJc9Ad'})

    # id_review, username, review, rate, date = [],[],[],[],[]

    id_r = [r.find('button', {'class':'PP3Y3d S1qRNe'})['data-review-id'] for r in review_list]
    # id_review.append(id_r)

    user = [r.find('div', class_='d4r55').find('span').text for r in review_list]
    # username.append(user)

    try:
        r_text = [r.find('span', class_='wiI7pd').text for r in review_list]
    except Exception:
        r_text = None
    # review.append(r_text)

    rating = [r.find('span', class_='kvMYJc')['aria-label'] for r in review_list]
    # rate.append(rating)

    rel_date = [r.find('span', class_='rsqaWe').text for r in review_list]
    # date.append(rel_date)

    review = pd.DataFrame(
        {'id': id_r,
        'nama': user,
         'rating': rating,
         'review': r_text,
         'waktu': rel_date})

    review.to_csv('perpusnas_google_review-2023_02_04_11_50_20.csv',index=False)
    print(review.shape)
#     print(review)

<ipython-input-8-3aad671ef00b>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=pathDriver, options=options)
<ipython-input-8-3aad671ef00b>:19: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  recent_rating_bt = driver.find_elements_by_xpath('//*[@id="action-menu"]/div[2]')[0]


Section 1 tahun lalu


<ipython-input-8-3aad671ef00b>:35: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  ele = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
<ipython-input-8-3aad671ef00b>:45: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  filter_date = driver.find_elements_by_class_name('rsqaWe')


scraping proccess


<ipython-input-8-3aad671ef00b>:62: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  item = driver.find_elements_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]')


(1040, 5)
Section 2 tahun lalu
scraping proccess
(1130, 5)


In [9]:
path = 'perpusnas_google_review-2023_02_04_08_50_20.csv'
df = pd.read_csv(path)

In [10]:
df

,id,nama,rating,review,waktu
0,ChZDSUhNMG9nS0VJQ0FnSUNocUlTV1ZBEAE,Annisa Vira,5 bintang,baguss,37 menit lalu
1,ChdDSUhNMG9nS0VJQ0FnSUNoaUlLNnpBRRAB,Intan Devi Puspita,5 bintang,"Cocok banget buat tempat belajar, tempat baca....",10 jam lalu
2,ChZDSUhNMG9nS0VJQ0FnSUNoOEplckJ3EAE,Anisya Chairunisyah,5 bintang,👍👍👍👍👍👍👍👍 …,11 jam lalu
3,ChZDSUhNMG9nS0VJQ0FnSUNoc0l6TVlnEAE,Christofer Nathanael,5 bintang,NaN,17 jam lalu
4,ChZDSUhNMG9nS0VJQ0FnSUNoMEtlUUtREAE,M. Azis Rizky Lubis,5 bintang,NaN,19 jam lalu
...,...,...,...,...,...
1125,ChZDSUhNMG9nS0VJQ0FnSURPcTh5V2JBEAE,Kadek Frisca Sarinastiti,5 bintang,NaN,6 bulan lalu
1126,ChdDSUhNMG9nS0VJQ0FnSURPcS1TX2hRRRAB,Rafi Ahmadprabowo,5 bintang,NaN,6 bulan lalu
1127,ChdDSUhNMG9nS0VJQ0FnSURPeTRHdWpRRRAB,Rucitra Deasy,5 bintang,Cozy place but can’t stand the lift queueing l...,6 bulan lalu
1128,ChdDSUhNMG9nS0VJQ0FnSURPeS1qWGdRRRAB,Nurul Laily Eprillya,5 bintang,NaN,6 bulan lalu


In [11]:
df['review'].isna().sum()

635

In [20]:
reviews = df[df['review'].notnull()].reset_index()

In [21]:
reviews

,index,id,nama,rating,review,waktu
0,0,ChZDSUhNMG9nS0VJQ0FnSUNocUlTV1ZBEAE,Annisa Vira,5 bintang,baguss,37 menit lalu
1,1,ChdDSUhNMG9nS0VJQ0FnSUNoaUlLNnpBRRAB,Intan Devi Puspita,5 bintang,"Cocok banget buat tempat belajar, tempat baca....",10 jam lalu
2,2,ChZDSUhNMG9nS0VJQ0FnSUNoOEplckJ3EAE,Anisya Chairunisyah,5 bintang,👍👍👍👍👍👍👍👍 …,11 jam lalu
3,14,ChdDSUhNMG9nS0VJQ0FnSURCNy1hMnJ3RRAB,Dtrans Tour Travel,5 bintang,Good place for to learn heritage\n\n(Diterjema...,2 hari lalu
4,17,ChZDSUhNMG9nS0VJQ0FnSURCdDllcFVnEAE,Vindita Tamara,5 bintang,Tempat keren buat healing dan edukasi keluarga,3 hari lalu
...,...,...,...,...,...,...
490,1115,ChdDSUhNMG9nS0VJQ0FnSURXaXBqT3RRRRAB,Fristiana “Tazmaniagirls” Natalia,5 bintang,Good vibes\nTenang dan bersih\nNyaman karena s...,6 bulan lalu
491,1119,ChZDSUhNMG9nS0VJQ0FnSURPLV82YUVnEAE,Alexa,5 bintang,(Diterjemahkan oleh Google) Lantai 12 bagus un...,6 bulan lalu
492,1123,ChZDSUhNMG9nS0VJQ0FnSURPNjl5WE5nEAE,Citra Anindita Pertiwi,5 bintang,Nyaman bgt buat ngerjain tugas atau sekedar ma...,6 bulan lalu
493,1127,ChdDSUhNMG9nS0VJQ0FnSURPeTRHdWpRRRAB,Rucitra Deasy,5 bintang,Cozy place but can’t stand the lift queueing l...,6 bulan lalu


In [22]:
reviews['review'][0:10]

0                                               baguss
1    Cocok banget buat tempat belajar, tempat baca....
2                                           👍👍👍👍👍👍👍👍 …
3    Good place for to learn heritage\n\n(Diterjema...
4       Tempat keren buat healing dan edukasi keluarga
5    liftnya crowded parah kalo siang/sore lumayan ...
6    Wonderfull place for learn and know your count...
7    Biasanya aku kesini kalo lagi bete dan galau a...
8                                   Tempatnya bagus!!!
9    Fasilitasnya yang harus di perbaiki liftnya si...
Name: review, dtype: object

In [33]:
from googletrans import Translator 

#create our translator object 
translator = Translator()
for ulas in reviews['review'][259:]:
    try:
        x = translator.translate(ulas, src='auto', dest='en').text
        en_reviews.append(x)
    except:
        break

In [34]:
len(en_reviews)

495

In [35]:
reviews['review'][422]

'Tempat nya nyaman ...banyak buku buku menarik untuk anak'

In [36]:
en_reviews[422]

'The place is comfortable ... many interesting books for children'

In [37]:
reviews['en_reviews'] = en_reviews

In [38]:
reviews

,index,id,nama,rating,review,waktu,en_reviews
0,0,ChZDSUhNMG9nS0VJQ0FnSUNocUlTV1ZBEAE,Annisa Vira,5 bintang,baguss,37 menit lalu,good
1,1,ChdDSUhNMG9nS0VJQ0FnSUNoaUlLNnpBRRAB,Intan Devi Puspita,5 bintang,"Cocok banget buat tempat belajar, tempat baca....",10 jam lalu,"Very suitable for a place to learn, place to r..."
2,2,ChZDSUhNMG9nS0VJQ0FnSUNoOEplckJ3EAE,Anisya Chairunisyah,5 bintang,👍👍👍👍👍👍👍👍 …,11 jam lalu,👍👍👍👍👍👍👍👍 …
3,14,ChdDSUhNMG9nS0VJQ0FnSURCNy1hMnJ3RRAB,Dtrans Tour Travel,5 bintang,Good place for to learn heritage\n\n(Diterjema...,2 hari lalu,Good place for to learn heritage\n\n(Translate...
4,17,ChZDSUhNMG9nS0VJQ0FnSURCdDllcFVnEAE,Vindita Tamara,5 bintang,Tempat keren buat healing dan edukasi keluarga,3 hari lalu,Cool place for healing and family education
...,...,...,...,...,...,...,...
490,1115,ChdDSUhNMG9nS0VJQ0FnSURXaXBqT3RRRRAB,Fristiana “Tazmaniagirls” Natalia,5 bintang,Good vibes\nTenang dan bersih\nNyaman karena s...,6 bulan lalu,Good Vibes\nCalm and clean\nComfortable becaus...
491,1119,ChZDSUhNMG9nS0VJQ0FnSURPLV82YUVnEAE,Alexa,5 bintang,(Diterjemahkan oleh Google) Lantai 12 bagus un...,6 bulan lalu,(Translated by google) 12th floor is good for ...
492,1123,ChZDSUhNMG9nS0VJQ0FnSURPNjl5WE5nEAE,Citra Anindita Pertiwi,5 bintang,Nyaman bgt buat ngerjain tugas atau sekedar ma...,6 bulan lalu,Very comfortable to do assignments or just pla...
493,1127,ChdDSUhNMG9nS0VJQ0FnSURPeTRHdWpRRRAB,Rucitra Deasy,5 bintang,Cozy place but can’t stand the lift queueing l...,6 bulan lalu,Cozy place but can’t stand the elevator queuei...
